# Correlated Subqueries

### Introduction

In this lesson, we'll learn about correlated subqueries.  With correlated subqueries, we first return a set of rows from an outer table.  And then our subquery makes a new query for each one of these rows to find a match.  

Let's see this by way of example.

### Loading our data

In [5]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('users.db')

table_names = ['employees', 'orders', 'customers', 'dog_foods']
root_url = "https://raw.githubusercontent.com/tech-interviews-jigsaw/sql-advanced-joins/main/6-common-strategies"
dfs = [pd.read_csv(f"{root_url}/{table_name}.csv") for table_name in table_names]

In [5]:
[df.to_sql(table_name, conn, index = True, index_label = 'id', if_exists = 'replace') for df, table_name in zip(dfs, table_names)]

[5, 3, 5, 5]

### Correlated Subqueries

With a correlated subquery, the query is executed once per each row in the outer query.  Let's see this by way of example.

Below we have a list of employees, with each employee assigned to a department.

In [131]:
query = """
select * from employees limit 2
"""

pd.read_sql(query, conn)

,id,employee_name,department,salary
0,0,Alice,HR,50000
1,1,Bob,IT,60000


Let's say that we want to find the highest salary for each department.  One way to do this is with a window function. 

In [7]:
query = """SELECT employee_name, department, salary,
       MAX(salary) OVER (PARTITION BY department ORDER BY salary) AS max_dept_salary
FROM employees e1"""
pd.read_sql(query, conn)

,employee_name,department,salary,max_dept_salary
0,Eve,Finance,62000,62000
1,Alice,HR,50000,50000
2,David,HR,52000,52000
3,Carol,IT,55000,55000
4,Bob,IT,60000,60000


But for this example, we'll use a correlated subquery.

To use a subquery, you start with just a normal query and give the table an alias.

In [12]:
query = """SELECT employee_name, department, salary
FROM employees e1"""
pd.read_sql(query, conn)

,employee_name,department,salary
0,Alice,HR,50000
1,Bob,IT,60000
2,Carol,IT,55000
3,David,HR,52000
4,Eve,Finance,62000


Then we add a subquery that -- for each row in that outer query -- performs another query.  

Below for each row, our subquery joins on the department name and finds the max salary.

In [ ]:
query = """SELECT employee_name, department, salary,
(SELECT MAX(salary) FROM employees e2
    WHERE e2.department = e1.department) AS max_dept_salary
FROM employees e1"""
pd.read_sql(query, conn)

Let's break down that correlated subquery one more time.  It consists of two components -- (1) the outer query and (2) the correlated subquery.

```sql
SELECT employee_name, department, salary,
(SELECT MAX(salary) FROM employees e2 WHERE e2.department = e1.department) AS max_sal -- 2. correlated subquery
FROM employees e1 -- 1. outer query
```

* **Outer Query**: The outer query retrieves rows from a table.  Notice that we immediately alias the outer query (e1).
* **Correlated Subquery**: Our subquery references the outer query, executing once per each row in the outer query.
    * Notice that the correlated subquery uses the outer table for filtering or calculation.

So really the way to think about a correlated subquery is as a loop.  We loop through all of the rows in our outer query, executing the correlated subquery once for each row.

### Moving to a use case

Ok, so as we said, we can achieve the above example with a window function -- where we display the max salary per department along with each row.

But what if we want to retrieve the *second* highest salary per department along with each row.  Here, our correlated subquery is more necessary.

You can see the query below.

Notice that this time in our subquery, we still join on the department, but we order by salary desc and apply an offset and limit to get the second highest salary.

In [13]:
query = """SELECT employee_name, department, salary,
(SELECT salary FROM employees e2 WHERE e2.department = e1.department
order by salary desc limit 1 offset 1 ) AS second_highest
FROM employees e1 order by department desc, salary desc"""
pd.read_sql(query, conn)

,employee_name,department,salary,second_highest
0,Bob,IT,60000,55000.0
1,Carol,IT,55000,55000.0
2,David,HR,52000,50000.0
3,Alice,HR,50000,50000.0
4,Eve,Finance,62000,NaN


Done.

So here, the correlated subquery appears better than the window function.  

> The closest we can do with a window function is calcuate the rank of the salary per department, and then select based on that.  But that's not exactly the sample.

In [135]:
query = """
with ranked_employees as (
    SELECT employee_name, department, salary,
     DENSE_RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS dept_rank
    FROM  employees
)
select * from ranked_employees where dept_rank = 2
"""
pd.read_sql(query, conn)

,employee_name,department,salary,dept_rank
0,Alice,HR,50000,2
1,Carol,IT,55000,2


### Two tables

So far we have use correlated subqueries when we are querying the same table.  But we can also perform correlated subqueries with multiple tables.

So let's say that we have customers and orders.  One way that we can find the those customers who have placed orders is with a correlated subquery.

> Yes, we can also do so with a simple inner join, but let's use a correlated subquery to get more familiar with it.

First we can take a look at our orders, where we can see that customers 1 and 2 have placed orders.

In [136]:
pd.read_sql("""select * from orders""", conn)

,id,order_id,customer_id,order_date
0,0,101,1,2023-01-15
1,1,102,1,2023-02-10
2,2,103,2,2023-03-05


Then we can use our correlated subquery.

In [137]:
query = """SELECT name
FROM customers c
WHERE (
    SELECT 1
    FROM orders o
    WHERE o.customer_id = c.customer_id
)"""
pd.read_sql(query, conn)

,name
0,John Smith
1,Jane Doe


We'll move onto a better use case in a second.  But for now, let's make sure we have the correlated subquery down.  Once again, we have our outer table `customers`, which we immediately alias as `c`.  Then for each row, we join our customers table on the orders table, and the final result only returns the rows where the correlated subquery returns a truthy value.

> Notice that for this last example, our correlated subquery is in the where clause.  We'll move back to using a correlated subquery to generate a new column of data in the next example.

### A better use case

Ok, so the above seems pretty complicated for something we can achieve with an inner join.  Now let's move towards a case where a correlated subquery is needed. 

Let's say that we have customers, each that has an available budget, and a list of dog foods.  Now  each customer wants to spend as much as possible on their dog.  So let's say that for each customer, we want to pair them with the most expensive dog food that is lower than their budget.   

Let's get to it.  

First, we'll look at our dog food data.

In [145]:
pd.read_sql("""select * from dog_foods""", conn)

,id,brand,price
0,0,Acme Dog Food,22
1,1,Puppy Chow,32
2,2,Healthy Paws,38
3,3,Bark Bites,19
4,4,Superior K9,45


And our customer budgets.

In [146]:
pd.read_sql("""select * from customers""", conn)

,id,customer_id,name,budget
0,0,1,John Smith,25
1,1,2,Jane Doe,30
2,2,3,Michael Brown,40
3,3,4,Emily Johnson,22
4,4,5,David Lee,50


And then we can get to the resultant query.

In [143]:
query = """SELECT *, 
(select dog_foods.price 
from dog_foods where dog_foods.price <= c.budget
order by price desc limit 1) as dog_food_price
FROM customers c"""
pd.read_sql(query, conn)

,id,customer_id,name,budget,dog_food_price
0,0,1,John Smith,25,22
1,1,2,Jane Doe,30,22
2,2,3,Michael Brown,40,38
3,3,4,Emily Johnson,22,22
4,4,5,David Lee,50,45


Let's copy this query below.

```sql
SELECT *, 
(select dog_foods.price 
from dog_foods where dog_foods.price <= c.budget
order by price desc limit 1) as dog_food_price
FROM customers c
```

So above, we first select all of our customers in the outer query.  Then in our subquery, for each customer, we find the dog food price that is lower than or equal to the budget.  Then we sort these from highest to lowest, returning the highest food price within this upper bound.    

### Summary

In this lesson, we saw how to perform a correlated subquery.  To perform a correlated subquery, we need an outer table, and a subquery that references that outer table.  

The correlated subquery executes for each row in the outer query, for either a filter or calculation. 

```sql
SELECT employee_name, department, salary,
(SELECT MAX(salary) FROM employees e2 WHERE e2.department = e1.department) AS max_sal -- 2. correlated subquery
FROM employees e1 -- 1. outer query
```

We also saw some use cases for our correlated subquery.  

For example, we saw saw a query that returns the second highest salary per department -- which goes further than what we can do with a window function.

In [ ]:
query = """SELECT employee_name, department, salary,
(SELECT salary FROM employees e2 WHERE e2.department = e1.department
order by salary desc limit 1 offset 1 ) AS max_sal
FROM employees e1 order by department desc, salary desc"""

And then finally, we saw how we can join two tables, aligning the rows based on our subquery -- as we did by finding the priciest dog food within each customer's budget.

### Resources

[Window Fn vs Subqueries](https://www.linkedin.com/pulse/comparing-sql-subqueries-window-functions-differences-siva-kowsika/)